In [1]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, HTML
import tickers_dexcalibur
import tickers_moonlight
import strategy_moonlight
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import threading

In [2]:
moonlight = {}
moonlight_master = pd.DataFrame()
backtest = False

def work():
    global backtest
    
    while backtest:
        button_coin.button_style = 'success'
        coin.options = list(tickers_moonlight.tickers.keys()) if w2.value == 1 else list(tickers_dexcalibur.tickers.keys())
        time.sleep(0.5)

        
def click_view(v):
    global moonlight
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.grid(True)
    ax.plot(moonlight[coin.value].index, moonlight[coin.value].BALANCE, color='C0')
    
    
def click_start(b):
    global w1, w2, moonlight, moonlight_master, backtest
    
    btc = 0.01
    fee = 0.001
    slippage = 0.001
    tickers = tickers_moonlight.tickers if w2.value == 1 else tickers_dexcalibur.tickers
    n_coins = len(tickers)
    
    moonlight = strategy_moonlight.get_data_bot(14,7,int(w1.value),tickers,btc,fee,slippage)
    
    moonlight_master = moonlight[list(moonlight.keys())[0]].copy()
    moonlight_master.POSITION = 0
    moonlight_master.BALANCE = 0
    moonlight_master.OPERATIONS = 0
    for coin_dataframe in moonlight:
        moonlight_master += moonlight[coin_dataframe]
    
    backtest = True
    # set up plot
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.grid(True)
    ax.plot(moonlight_master.index, moonlight_master.BALANCE, color='C0')
    
    thread = threading.Thread(target=work)
    thread.start()

In [3]:
display(HTML("<h3>REPORT BOT ACADEMY tools</h3>"))

w1 = widgets.Dropdown(
    options=['30', '60', '90', '120'],
    value='90',
    description='BACKTEST:',
    disabled=False,
)

w2 = widgets.Dropdown(
    options=[('MOONLIGHT', 1), ('DEXCALIBUR', 2)],
    value=1,
    description='STRATEGY:',
)

button_start = widgets.Button(
    description='START',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='START',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)

coin = widgets.Dropdown(
    options=list(tickers_moonlight.tickers.keys()),
    description='COIN:',
)

button_coin = widgets.Button(
    description='VIEW',
    disabled=False,
    button_style='info' if backtest else 'warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='VIEW',
    icon='chart-line' # (FontAwesome names without the `fa-` prefix)
)

button_start.on_click(click_start)
button_coin.on_click(click_view)

widgets.HBox([widgets.VBox([w1, w2, button_start]),widgets.HBox([coin, button_coin])])